In [1]:
import sklearn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import ParameterGrid
from sklearn.linear_model import LogisticRegression
# from lightgbm import LGBMClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from operator import itemgetter 
import os
from sklearn.metrics import f1_score
from tqdm import trange

In [2]:
from sklearn.datasets import load_diabetes
X,Y =load_diabetes(return_X_y=True)

In [3]:
os.cpu_count()

4

In [4]:
class compare_classifiers():
    def __init__(self, 
                X: '(np ndarray) size of num_samplesxnum_features', 
                Y: '(np ndarray[int]) size of num_samples', 
                n_folds: '(int)',
                random_state=456):
        self.x = X
        self.y = Y
        self.n_folds = n_folds
        self.kfold = StratifiedKFold(n_splits=self.n_folds, shuffle=True, random_state=random_state)
        
    def all_models(self):
        param_grid = ParameterGrid({'C': [0.01, 0.1, 1, 10, 100], 'solver': ['lbfgs', 'liblinear']})
        for param in param_grid:
            model = LogisticRegression(class_weight='balanced', multi_class='ovr')
            model.set_params(**param)
            yield {'model':model, 'model_name':'Logistic', 'param': param}

#         param_grid = ParameterGrid({'num_leaves': [4, 8, 32, 64], 'solver' = ['lbfgs', 'liblinear']})
#         for param in param_grid:
#             model = LGBMClassifier(class_weight='balanced', solver= , objective='multiclass')
#             model.set_params(**param)
#             yield (model, 'linear')
        
        param_grid = ParameterGrid({'C': [0.01, 0.1, 1, 10, 100]})
        for param in param_grid:
            model = SVC(class_weight='balanced',  kernel = 'linear')
            model.set_params(**param)
            yield {'model':model, 'model_name':'SVM_linear', 'param': param}
        
        param_grid = ParameterGrid({'C': [0.01, 0.1, 1, 10, 100]})
        for param in param_grid:
            model = SVC(class_weight='balanced',  kernel = 'rbf')
            model.set_params(**param)
            yield {'model':model, 'model_name':'SVM_rbf', 'param': param}
        
        param_grid = ParameterGrid({'criterion': ["gini", "entropy"], 
                                    'max_depth': [4, 12, 16, 24]})
        for param in param_grid:
            model = RandomForestClassifier(class_weight='balanced', max_features='log2', n_jobs=3)
            model.set_params(**param)
            yield {'model':model, 'model_name':'RandomForest', 'param': param}
            
        model = GaussianNB()
        yield {'model':model, 'model_name':'GaussianNB', 'param': None}
    def fit_models(self):
        across_fold_UWf1s = []
        across_fold_Wf1s = []
        model_list = []
        for ind, (train, test) in enumerate(self.kfold.split(self.x, self.y)):
            print(f'fold: {ind}')
            UWf1s = []
            Wf1s = []
            print(train)
            print(test)
            x_tr = self.x[train]
            y_tr = self.y[train]
            x_te = self.x[test]
            y_te = self.y[test]
            print('begin_training')
            for model_dict in self.all_models():
                print('fit model')
                print(model_dict['model_name'])
                print(model_dict['param'])
                model_dict['model'].fit(x_tr, y_tr)
                print('after fitting')
                y_pred = model_dict['model'].predict(x_te)
                print('after predicting')
                UWf1s.append(f1_score(y_te, y_pred, average='macro'))
                Wf1s.append(f1_score(y_te, y_pred, average='weighted'))
                if ind == 0:
                    model_list.append(model_dict['model_name'])
            across_fold_UWf1s.append(UWf1s)
            across_fold_Wf1s.append(Wf1s)
            print(across_fold_UWf1s)
            print(across_fold_Wf1s)
        print('1')
        across_fold_UWf1s = np.stack(across_fold_UWf1s)
        across_fold_Wf1s = np.stack(across_fold_Wf1s)
        across_fold_UWf1s = np.mean(across_fold_UWf1s,axis=0)
        across_fold_Wf1s = np.mean(across_fold_Wf1s,axis=0)
        f1s = (across_fold_UWf1s+across_fold_Wf1s)/2
        best_dictionary = {}
        print(len(model_list))
        print(len(f1s))
        for ind, (model_name, f1) in enumerate(zip(model_list, f1s)):
            if model_name not in best_dictionary:
                best_dictionary[model_name] = (ind, f1)
            else:
                if best_dictionary[model_name][1] < f1:
                    best_dictionary[model_name] = (ind, f1)
        for model_name in best_dictionary:
            best_dictionary[model_name] = {'Unweighted_f1': across_fold_UWf1s[best_dictionary[model_name][0]],
                                          'Weighted_f1': across_fold_Wf1s[best_dictionary[model_name][0]],
                                          'Avg_f1': f1s[best_dictionary[model_name][0]]}
        model_names = []
        Unweighted_f1 = []
        Weighted_f1 = []
        Avg_f1 = []
        print('2')
        for model_name in best_dictionary:
            model_names.append(model_names)
            Unweighted_f1.append(best_dictionary[model_name]['Unweighted_f1'])
            Weighted_f1.append(best_dictionary[model_name]['Weighted_f1'])
            Avg_f1.append(best_dictionary[model_name]['Avg_f1'])
        print('3')
        df = pd.DataFrame({'Model': model_names, 'Unweighted_f1score': Unweighted_f1, 
                           'Weighted_f1score':Weighted_f1, 'Avg_f1score': Avg_f1})
        print(df)

In [5]:
a = compare_classifiers(X,Y,n_folds=5)

In [6]:
X.shape

(442, 10)

In [ ]:
a.fit_models()

/home/kamin/.virtualenv/dev36/lib/python3.6/site-packages/sklearn/model_selection/_split.py:672: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)


fold: 0
[  1   3   4   5   6   7   8  10  12  13  14  15  16  17  19  20  21  23
  24  25  26  27  29  31  32  33  35  36  38  39  40  41  43  44  45  48
  49  50  51  52  53  54  55  56  58  59  60  61  62  65  66  67  68  69
  70  71  72  73  74  75  76  77  78  80  81  82  83  84  85  86  87  88
  89  91  92  93  94  95  96  97  98  99 100 102 104 105 106 108 109 110
 112 113 115 116 117 118 119 120 121 123 125 126 127 128 129 130 131 132
 133 134 137 138 139 141 142 143 144 146 147 149 150 152 153 154 155 156
 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174
 175 176 177 179 180 182 184 185 186 187 188 189 190 191 193 194 196 197
 198 199 201 203 204 205 206 207 208 209 211 212 213 214 215 216 217 219
 221 222 223 225 226 227 229 230 232 234 235 236 237 238 239 240 241 242
 245 246 248 249 250 252 253 254 255 256 257 258 260 261 262 263 264 265
 266 268 269 274 276 277 278 279 280 281 282 283 285 286 288 289 290 291
 292 294 295 296 297 298 300 301 302 303 30

In [11]:
def aaa(kwangs):
    print(kwangs)

In [12]:
a= {'kernel': 'rbf', 'gamma':  10}
aaa(a)

{'kernel': 'rbf', 'gamma': 10}


In [77]:
Y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [18]:
X.shape

(150, 4)

In [38]:
Y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [29]:
os.getcwd()

'/home/kamin/Thai_NLP'

In [30]:
os.chdir('..')

In [31]:
import pickle

In [33]:
with open('result_123.pkl','rb') as f:
    out = pickle.load(f)

In [34]:
out

{'f1s': {'negative': 0.7557251908396946,
  'neutral': 0.7948717948717949,
  'positive': 0.9324324324324325},
 'weighted_f1': 0.8095179353769782,
 'unweighted_f1': 0.8276764727146406,
 'conf_mat': array([[ 99,  30,   3],
        [ 29, 124,   0],
        [  2,   5,  69]])}